In [4]:
import smtplib
import pandas as pd
import os
import datetime
import pytz
import time

class AutoEmailSend():
    def __init__(self):
        self.csv_file_path = r'/content/Sample.csv' # provide csv path here
        self.f = open('text.txt','w')
        self.user = 'dummy.nl4@gmail.com'
        self.password = 'Dummy@123'
        self.email_id = 'dummy.nl4@gmail.com'
        
        self.df_metadata = pd.read_csv(self.csv_file_path)  
        self.email = str()
        self.message = str()
        self.phone_number = str()
        self.country = str()
        self.date = str()
        self.message_status = False
        self.phone_status = False
        self.email_status = False
        self.date_status = False
        self.zone = None

    def send_email(self):
        for index, df_data in self.df_metadata.iterrows():
            self.email = str(df_data['Email'])
            self.message = str(df_data['Message'])
            self.phone_number = str(df_data['Phone'])
            self.country = str(df_data['Country'])
            self.date = str(df_data['Schedule On'])
            self.message_status = self.check_message()
            self.phone_status, self.phone_number = self.check_phone_number()
            self.email_status = self.check_email()
            self.date_status, self.date = self.check_datetime()
            
            if self.message_status * self.phone_status * self.email_status * self.date_status:
                self.datetime_for_country()
                #self.send_email_at()
            
            
        
            
    def check_message(self): 
        self.message_status = False
        message_length = len(self.message)
        if message_length > 1 and message_length <=160:
            self.message_status = True     
        else:
            with open('text.txt', 'w') as f:
              f.writelines('Error 1 : Message length is {} , which improper.'.format(message_length))    
        return self.message_status

    def check_phone_number(self):
        self.phone_status = False
        try:
            if len(self.phone_number) == 10:
                self.phone_number = int(self.phone_number)
                self.phone_status = True
                return self.phone_status, self.phone_number
            else:
                with open('text.txt', 'a') as f:
                  f.writelines('\nError 2 : Phone number length is {} , which improper.'.format(len(self.phone_number))) 
                return self.phone_status, 0
        except Exception as err:
            with open('text.txt', 'a') as f:
              f.writelines('\nError 3 : Phone number {} is improper.'.format(self.phone_number))
            return self.phone_status, 0

    def check_email(self):
        self.email_status = False
        email_split = self.email.split('@')
        if len(email_split) == 2:
            if email_split[-1] in ['gmail.com', 'yahoo.co.in', 'email.in']:
                self.email_status = True
                return self.email_status
            else:
              with open('text.txt', 'a') as f:
                f.writelines('\nError 4 : Domain of Email {} is improper.'.format(self.email))
            return self.email_status
               
        else:
          with open('text.txt', 'a') as f:
            f.writelines('\nError 5 : Email {} is improper.'.format(self.email))
            return self.email_status

    def check_datetime(self):
        self.date_status = False
        if len(self.date) > 0:
            try:
                self.date = datetime.datetime.strptime(self.date, '%d-%m-%Y')
                self.date += datetime.timedelta(minutes=10*60)
                self.date_status = True
                return self.date_status, self.date
            except Exception as err:
              with open('text.txt', 'a') as f:
                f.writelines('\nError 6 : Date format {} is improper.'.format(self.date))
                return self.date_status, 0
        else:
            return self.date_status, 0
        
    def smtp_email(self):
        try:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(self.user, self.Dummy@123)
            server.sendmail(self.email_id, self.email, self.message)
            server.quit()
            print('Email sent Successful')
        except SMTPException as err:
          with open('text.txt', 'a') as f:
            f.writelines('\nError 7 : Failed to sent email. Error is {}'.format(err))
    
    def datetime_for_country(self):
        if self.country == 'INDIA':
            self.zone = pytz.timezone('Asia/Kolkata')
        elif self.country == 'USA':
            self.zone = pytz.timezone('US/Central')
        else:
            self.zone = pytz.timezone('GMT')
        self.date = self.zone.localize(self.date)
        
    # def send_email_at(self):
    #     time.sleep(self.date.timestamp() - datetime.datetime.now(self.zone))
    #     self.smtp_email()
    #     with open('text.txt', 'w') as f:
    #       f.writelines('Job Done for mail {}'.format(self.email))
        
        

if __name__ == "__main__":   
     EmailSend = AutoEmailSend()   
     EmailSend.send_email()
     with open('text.txt','a') as d:
        if(os.stat("text.txt").st_size == 0):
          d.writelines('Success')
     